In [12]:
#识别手写数字
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from visdom import Visdom

In [13]:
batch_size=200
learning_rate=0.01
epochs=20

In [14]:
# 获取训练和测试数据
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),#将PIL图像转换为Tensor，并自动归一化到[0,1]范围
                       # transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=True)

In [15]:
# 定义多层感知机（全连接网络）
class MLP(nn.Module):

    def __init__(self):
        super(MLP, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(784, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 200),
            nn.LeakyReLU(inplace=True),
            nn.Linear(200, 10),
            nn.LeakyReLU(inplace=True),
        )

    def forward(self, x):
        x = self.model(x)

        return x

# 定义训练过程
device = torch.device('cpu')
net = MLP().to(device)
optimizer = optim.SGD(net.parameters(), lr=learning_rate)
criteon = nn.CrossEntropyLoss().to(device)

In [16]:
# 定义两个用于可视化训练和测试过程visdom窗口，即两张图
viz = Visdom()

viz.line([0.], [0.], win='train_loss', opts=dict(title='训练集损失'))
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='测试集损失和准确率', legend=['损失', '准确率']))
global_step = 0

Setting up a new session...


In [17]:
# 开始训练，并给图送入实时更新的数据，以可视化训练过程
for epoch in range(epochs):

    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(-1, 28*28)
        data, target = data.to(device), target  #target.cuda()

        logits = net(data)
        loss = criteon(logits, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        global_step += 1
        # 给‘train_loss’送入数据
        viz.line([loss.item()], [global_step], win='train_loss', update='append')

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))


    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.view(-1, 28 * 28)
        data, target = data.to(device), target    #target.cuda()
        logits = net(data)
        test_loss += criteon(logits, target).item()

        pred = logits.argmax(dim=1)#获取预测类别
        correct += pred.eq(target).float().sum().item()
    
    # 给‘test’送入数据
    viz.line([[test_loss, correct / len(test_loader.dataset)]],
             [global_step], win='test', update='append')#更新方式
    # 可视化当前测试的数字图片
    viz.images(data.view(-1, 1, 28, 28), win='x')
    # 可视化测试结果
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
             opts=dict(title='pred'))

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.306792
Train Epoch: 0 [20000/60000 (33%)]	Loss: 2.280395
Train Epoch: 0 [40000/60000 (67%)]	Loss: 2.250000

Test set: Average loss: 0.0111, Accuracy: 4649.0/10000 (46%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.225038
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.155950
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.062370

Test set: Average loss: 0.0097, Accuracy: 4697.0/10000 (47%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.934498
Train Epoch: 2 [20000/60000 (33%)]	Loss: 1.789672
Train Epoch: 2 [40000/60000 (67%)]	Loss: 1.787987

Test set: Average loss: 0.0079, Accuracy: 5499.0/10000 (55%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 1.617782
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.461216
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.421934

Test set: Average loss: 0.0068, Accuracy: 5842.0/10000 (58%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.381685
Train Epoch: 4 [20000/60000 (33%)]	Loss: 1.133218
Train Epoch: 4 [40000/60000 (67%)]	Loss: 1.135623

Test set: Average loss